<a href="https://colab.research.google.com/github/ayseakinn/ManavUygulamasi1/blob/master/Veri_Temizleme_ve_Yorum_Analizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
from google.colab import drive
import numpy as np
import pandas as pd
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
!pip install simplemma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 7.2 MB/s eta 0:00:00


In [78]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

In [82]:
df = pd.read_excel("/content/gdrive/MyDrive/temizleme.xlsx")
etiketler = [0,1]
df = df[df.etiket.isin(etiketler)].copy() #sadece 1-0 olanlari kayit eder 2leri siler
df.to_excel('temzileme.xlsx', index=False) # bu dosyaya
df.head()

,Yorumlar,etiket
0,Yaaaa cidden son mu olacak yok artık böyle bi ...,0
1,Muhteşem bir film. Önceki yıllara gitmeleri ço...,1
2,film beklentileri karşılamadı. Thor ne öyle! ...,0
3,Güzel bir veda fakat filmi boş diyaloglarla uz...,0
4,Aksiyon son sahneye saklanmış. Sonu her ne ka...,1


In [96]:
stop_words = None
with open("stopwords.txt", "r") as stop_file:
    stop_words = set(stop_file.read().splitlines())

In [85]:

def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)

    text = " ".join(text.lower().strip().split())
    text = text_lemmatizer(text, lang="tr")

    return " ".join([word for word in text if word not in stop_words])

In [86]:
len(df)

5

In [ ]:
df["Yorumlar"] = df["Yorumlar"].astype(str)
df["etiket"] = df["Yorumlar"].apply(clean_text)

In [93]:
df.to_excel('temizveri.xlsx', index=False)

In [ ]:
import random
for i in range(3):
    rand_idx = random.randrange(len(df))
    ozet = df.iloc[rand_idx]["Yorumlar"]
    print("Orijinal:")
    print(ozet + "\n")
    print("Temizlenmiş:")
    print(df.iloc[rand_idx]["clean"])
    print("/" * 50 + "\n")

In [ ]:

import random
for i in range(3):
    rand_idx = random.randrange(len(df))
    ozet = df.iloc[rand_idx]["Metin"]
    print("Orijinal:")
    print(ozet + "\n")
    print("Temizlenmiş:")
    print(df.iloc[rand_idx]["clean"])
    print("/" * 50 + "\n")


In [35]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import pandas as pd

In [80]:
df = pd.read_excel('/content/gdrive/MyDrive/Yeni Microsoft Excel Çalışma Sayfası.xlsx')
df.head()
display(pd.DataFrame(df))


,Yorumlar,etiket,Clean
0,Yaaaa cidden son mu olacak yok artık böyle bi ...,0,Sonu böyle bitemez
1,Muhteşem bir film. Önceki yıllara gitmeleri ço...,1,Muhteşem film
2,film beklentileri karşılamadı. Thor ne öyle! ...,0,Flim beklentileri karşılamadı
3,Güzel bir veda fakat filmi boş diyaloglarla uz...,0,Güzel ama uzatmışlar
4,Aksiyon son sahneye saklanmış. Sonu her ne ka...,1,Güzel vedaydı


In [67]:
X = df['Clean'].to_numpy()
y = df['etiket'].to_numpy()

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [69]:
print(len(X_train), len(X_test))

4 1


In [70]:
X_train[0]

'Güzel vedaydı'

In [71]:
vectorizer = TfidfVectorizer(input='content')

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [72]:
model_NB = MultinomialNB()
model_NB.fit(X_train, y_train)

MultinomialNB()

In [73]:
print("NB train accuracy:", model_NB.score(X_train, y_train))
print("NB test accuracy:", model_NB.score(X_test, y_test))

predictions_train = model_NB.predict(X_train)
print("NB Train F1:", f1_score(y_train, predictions_train))

predictions_test = model_NB.predict(X_test)
print("NB Test F1:", f1_score(y_test, predictions_test))

NB train accuracy: 0.75
NB test accuracy: 0.0
NB Train F1: 0.0
NB Test F1: 0.0


In [74]:

model_DT = DecisionTreeClassifier()
model_DT.fit(X_train, y_train)


DecisionTreeClassifier()

In [75]:

print("DT train accuracy:", model_DT.score(X_train, y_train))
print("DT test accuracy:", model_DT.score(X_test, y_test))

predictions_train = model_DT.predict(X_train)
print("DT Train F1:", f1_score(y_train, predictions_train))

predictions_test = model_DT.predict(X_test)
print("DT Test F1:", f1_score(y_test, predictions_test))

DT train accuracy: 1.0
DT test accuracy: 0.0
DT Train F1: 1.0
DT Test F1: 0.0


In [76]:
lyric = "ürün çalışıyor"
lyric_vec = vectorizer.transform([lyric])
# olumsuz: 0
# olumlu: 1
print("NB:", model_NB.predict(lyric_vec))
print("DT:", model_DT.predict(lyric_vec))

print("NB Proba:", model_NB.predict_proba(lyric_vec))
print("DT Proba:", model_DT.predict_proba(lyric_vec))

NB: [0]
DT: [0]
NB Proba: [[0.75 0.25]]
DT Proba: [[1. 0.]]
